In [82]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [83]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical

In [84]:
dataset = 'mag-lev-images'
size = 64

In [85]:
def class_label(img):
    if '100-lido' in img:
        lbl = 0
    elif '100-caff' in img:
        lbl = 1
    elif '50-50-lido-caff' in img:
        lbl = 2
    return lbl

In [86]:
def images_with_label():
    images = []
    for i in tqdm(os.listdir(dataset)):
        path = os.path.join(dataset, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (size, size))
            images.append([np.array(img), class_label(i)])
    return images

In [105]:
images = images_with_label()

X = np.array([i[0] for i in images]).reshape(-1, size, size, 1)
y = np.array([i[1] for i in images])

100%|██████████| 29/29 [00:00<00:00, 77.07it/s]


In [106]:
kfold = StratifiedKFold(n_splits=7, shuffle=True)
cvscores = []

In [ ]:
for train, test in kfold.split(X, y):    
    model = Sequential()

    model.add(InputLayer(input_shape=[size,size,1]))
    model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(3,activation='softmax'))
    optimizer = Adam(lr=1e-2)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(x=X[train],y=to_categorical(y[train]),epochs=50,batch_size=10)
    model.summary()
    
    # evaluate the model
    scores = model.evaluate(X[test], to_categorical(y[test]), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    print("\n\n")

Epoch 1/200
24/24 [==============================] - 1s 47ms/step - loss: 84.5724 - accuracy: 0.4583
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 2.1298 - accuracy: 0.4583
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1154 - accuracy: 0.3333
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1049 - accuracy: 0.3750
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1047 - accuracy: 0.3333
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 1.0952 - accuracy: 0.3333
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1004 - accuracy: 0.2917
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1090 - accuracy: 0.3333
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0889 - accuracy: 0.4167
Epoch 10/200
24/24 [==============================] - 0s 3ms/step - loss: 1.1162 - accuracy: 0.2917
Epoch 1

Epoch 83/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1145 - accuracy: 0.3750
Epoch 84/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1076 - accuracy: 0.3750
Epoch 85/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0861 - accuracy: 0.4167
Epoch 86/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0895 - accuracy: 0.4167
Epoch 87/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0822 - accuracy: 0.3750
Epoch 88/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1066 - accuracy: 0.3750
Epoch 89/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1026 - accuracy: 0.3750
Epoch 90/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1065 - accuracy: 0.3750
Epoch 91/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0962 - accuracy: 0.3333
Epoch 92/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1173 - accuracy: 0.3750


24/24 [==============================] - 0s 4ms/step - loss: 1.0892 - accuracy: 0.4167
Epoch 165/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0986 - accuracy: 0.2917
Epoch 166/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0801 - accuracy: 0.3750
Epoch 167/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0874 - accuracy: 0.3333
Epoch 168/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0944 - accuracy: 0.3750
Epoch 169/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1018 - accuracy: 0.3333
Epoch 170/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1069 - accuracy: 0.2917
Epoch 171/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1013 - accuracy: 0.3333
Epoch 172/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0973 - accuracy: 0.3333
Epoch 173/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0759 - accuracy: 0.4583
Epoc

24/24 [==============================] - 0s 4ms/step - loss: 1.1100 - accuracy: 0.2500
Epoch 28/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0963 - accuracy: 0.2083
Epoch 29/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1102 - accuracy: 0.3750
Epoch 30/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0766 - accuracy: 0.4167
Epoch 31/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1162 - accuracy: 0.3333
Epoch 32/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0867 - accuracy: 0.4167
Epoch 33/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1166 - accuracy: 0.4167
Epoch 34/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0878 - accuracy: 0.4167
Epoch 35/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0805 - accuracy: 0.3750
Epoch 36/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0850 - accuracy: 0.5000
Epoch 37/200


Epoch 109/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1013 - accuracy: 0.2917
Epoch 110/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0968 - accuracy: 0.2917
Epoch 111/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1057 - accuracy: 0.4167
Epoch 112/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0934 - accuracy: 0.3750
Epoch 113/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1005 - accuracy: 0.2500
Epoch 114/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0806 - accuracy: 0.4167
Epoch 115/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0854 - accuracy: 0.4167
Epoch 116/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0976 - accuracy: 0.3750
Epoch 117/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0938 - accuracy: 0.3750
Epoch 118/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0942 - accurac

...

Epoch 108/200
24/24 [==============================] - 0s 4ms/step - loss: 0.5835 - accuracy: 0.7083
Epoch 109/200
24/24 [==============================] - 0s 4ms/step - loss: 0.7892 - accuracy: 0.5000
Epoch 110/200
24/24 [==============================] - 0s 4ms/step - loss: 0.6088 - accuracy: 0.6250
Epoch 111/200
24/24 [==============================] - 0s 4ms/step - loss: 0.6994 - accuracy: 0.6250
Epoch 112/200
24/24 [==============================] - 0s 4ms/step - loss: 0.7171 - accuracy: 0.5833
Epoch 113/200
24/24 [==============================] - 0s 4ms/step - loss: 0.5317 - accuracy: 0.6667
Epoch 114/200
24/24 [==============================] - 0s 4ms/step - loss: 0.7644 - accuracy: 0.7500
Epoch 115/200
24/24 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.5833
Epoch 116/200
24/24 [==============================] - 0s 4ms/step - loss: 0.8981 - accuracy: 0.5833
Epoch 117/200
24/24 [==============================] - 0s 4ms/step - loss: 0.6752 - accurac

24/24 [==============================] - 0s 4ms/step - loss: 0.7610 - accuracy: 0.6250
Epoch 190/200
24/24 [==============================] - 0s 4ms/step - loss: 0.6291 - accuracy: 0.5417
Epoch 191/200
24/24 [==============================] - 0s 4ms/step - loss: 0.6572 - accuracy: 0.6250
Epoch 192/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1009 - accuracy: 0.5417
Epoch 193/200
24/24 [==============================] - 0s 4ms/step - loss: 1.3247 - accuracy: 0.4583
Epoch 194/200
24/24 [==============================] - 0s 4ms/step - loss: 1.2971 - accuracy: 0.4167
Epoch 195/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1699 - accuracy: 0.4167
Epoch 196/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0951 - accuracy: 0.2917
Epoch 197/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1102 - accuracy: 0.3750
Epoch 198/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0788 - accuracy: 0.4583
Epoc

24/24 [==============================] - 0s 4ms/step - loss: 1.0973 - accuracy: 0.3750
Epoch 53/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1115 - accuracy: 0.3750
Epoch 54/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0926 - accuracy: 0.2917
Epoch 55/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1111 - accuracy: 0.2917
Epoch 56/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0938 - accuracy: 0.3333
Epoch 57/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0673 - accuracy: 0.4167
Epoch 58/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1022 - accuracy: 0.3750
Epoch 59/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0975 - accuracy: 0.3750
Epoch 60/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0944 - accuracy: 0.2917
Epoch 61/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0888 - accuracy: 0.4583
Epoch 62/200


24/24 [==============================] - 0s 4ms/step - loss: 1.0873 - accuracy: 0.4583
Epoch 134/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1098 - accuracy: 0.4167
Epoch 135/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0908 - accuracy: 0.3750
Epoch 136/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1104 - accuracy: 0.3750
Epoch 137/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1103 - accuracy: 0.3750
Epoch 138/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1369 - accuracy: 0.3750
Epoch 139/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0860 - accuracy: 0.3750
Epoch 140/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0989 - accuracy: 0.3750
Epoch 141/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0730 - accuracy: 0.4167
Epoch 142/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0996 - accuracy: 0.3750
Epoc

accuracy: 25.00%



Epoch 1/200
24/24 [==============================] - 1s 48ms/step - loss: 132.7781 - accuracy: 0.3333
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 5.4356 - accuracy: 0.3750
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 1.6282 - accuracy: 0.3333
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0997 - accuracy: 0.3750
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 1.1019 - accuracy: 0.3750
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0920 - accuracy: 0.3750
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0914 - accuracy: 0.3750
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0825 - accuracy: 0.3750
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0706 - accuracy: 0.4167
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 1.0753 - acc

In [ ]:
print("Average Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))